Activity for 3NF & 4NF
------------

This activity will help you get familiar with some of the normal forms which are not covered in the lecture but you might come across them as you dive more into this field.

In this activity, you can use the following tools to test your solutions:

In [1]:
def to_set(x):
    if type(x) == set:
        return x
    elif type(x) in [list, set]:
        return set(x)
    elif type(x) in [str, int]:
        return set([x])
    else:
        raise Exception("Unrecognized type.")
        
def fd_to_str((lhs,rhs)): return ",".join(to_set(lhs)) + " -> " + ",".join(to_set(rhs))

def fds_to_str(fds): return "\n\t".join(map(fd_to_str, fds))

def set_to_str(x): return "{" + ",".join(x) + "}"

def fd_applies_to(fd, x): 
    lhs, rhs = map(to_set, fd)
    return lhs.issubset(x)

def compute_closure(x, fds, verbose=False):
    bChanged = True        # We will repeat until there are no changes.
    x_ret    = to_set(x).copy()    # Make a copy of the input to hold x^{+}
    while bChanged:
        bChanged = False   # Must change on each iteration
        for fd in fds:     # loop through all the FDs.
            (lhs, rhs) = map(to_set, fd) # recall: lhs -> rhs
            if fd_applies_to(fd, x_ret) and not rhs.issubset(x_ret):
                x_ret = x_ret.union(rhs)
                if verbose:
                    print("Using FD " + fd_to_str(fd))
                    print("\t Updated x to " + set_to_str(x_ret))
                bChanged = True
    return x_ret

def is_superkey_for(A, X, fds, verbose=False): 
    return X.issubset(compute_closure(A, fds, verbose=verbose))

import itertools
def is_key_for(A, X, fds, verbose=False):
    subsets = set(itertools.combinations(A, len(A)-1))
    return is_superkey_for(A, X, fds) and \
        all([not is_superkey_for(set(SA), X, fds) for SA in subsets])

## 3rd Normal Form (3NF)

As we have seen in the lecture (Lecture 5-7 slides 104-107) on BCNF that it is not possible, in some cases, to decompose a relation into BCNF relations that have both the lossless-join and dependency-preservation properties. 
The solution to this is to relax our BCNF requirement slightly, in order to allow the occasional relation schema that can not be decomposed into BCNF relations without our losing the ability to check the FD’s. This relaxed condition is called "Third Normal Form". 

A relation R is in 3rd Normal Form if:

&nbsp;&nbsp;&nbsp;&nbsp; If ∃ a nontrivial dependency A = {A1, A2, ..., An} -> B in R, <br>
&nbsp;&nbsp;&nbsp;&nbsp; then A is a superkey for R OR B is part of some key. 

You can read more about this in the section <b>3.5</b> of the textbook <b>Database Systems: The Complete Book by Garcia-Molina, Ullman, and Widom</b>.

### Exercise 1

Given the set of attributes below, and using the tools above to test / justify your conclusion, come up with the smallest number of FDs such that the table with attributes $A$ (below) is _in 3NF but **not BCNF**_:

In [4]:
A = set(['A','B','C'])

In [5]:
F = []

### 3NF Decomposition Algorithm

3NFDecomp(R):<br>
   &nbsp;&nbsp;&nbsp;&nbsp; <u>let</u> K = [all attributes that are part of some key]<br>
   &nbsp;&nbsp;&nbsp;&nbsp; Find A s.t.: <b>A<sup>+</sup> \ (A &cup; K) ≠ ∅</b> and <b>A<sup>+</sup> ≠ [all attributes]</b> <br>

   &nbsp;&nbsp;&nbsp;&nbsp; <u>if</u> (not found) <u>then</u> Return R <br>
   
   &nbsp;&nbsp;&nbsp;&nbsp; let <b>B = A<sup>+</sup> \ (A &cup; K)</b>,  <b>C = B<sup>C</sup> \ A</b> <br>
   &nbsp;&nbsp;&nbsp;&nbsp; decompose R into <b>R<sub>1</sub>(A &cup; B)</b> and <b>R<sub>2</sub>(A &cup; C)</b> <br>
   
   &nbsp;&nbsp;&nbsp;&nbsp; Return 3NFDecomp(R<sub>1</sub>), 3NFDecomp(R<sub>2</sub>)


### Exercise 2

Give the decomposition of the relation $R$ with attributes $A$ and FDs $F$ below in 3rd normal form, using the tools again to test / justify your steps:

In [ ]:
A = set(['A','B','C','D','E'])
F = [(set(['B','C']), 'D'),
     (set(['D']), 'E'),
     (set(['E']), 'C'),
     (set(['E']), 'A')]

## 4th Normal Form (4NF)

There are occasional situations where we design a relation schema and find it is in BCNF, yet the relation has a kind of redundancy that is not related to FD’s (see section 3.6.1 of the textbook). These redundancies are caused by the MVDs of the relation and can be eliminated if we also use these MVDs for decomposition. This is where we use a new normal form, called "Fourth Normal Form". 
In this normal form, all nontrivial MVD’s are eliminated, as are all FD’s that violate BCNF. As a result, the decomposed relations have neither the redundancy from FD’s nor the redundancy from MVD’s. The fourth normal form condition is essentially the BCNF condition, but applied to MVD’s instead of FD’s. Formally:

A relation R is in 4th Normal Form if:

&nbsp;&nbsp;&nbsp;&nbsp; If ∃ a nontrivial MVD, A = {A1, A2, ..., An}&#x21A0;B in R, <br>
&nbsp;&nbsp;&nbsp;&nbsp; then A is a superkey for R. 

You can read more about this normal form in the section <b>3.6.4</b> of the textbook.

### Exercise 3

Given the relation R with attributes A, MVDs M and FDs F below, check if R is in 4th Normal Form or not, using the tools again to test / justify your conclusion:

In [ ]:
A = set(['A','B','C','D','E'])
F = [(set(['A']), 'B'),
     (set(['B']), 'C')]
M = [(set(['A','B']), 'C'),
     (set(['D']), 'E')]

### 4NF Decomposition Algorithm

4NFDecomp(R):<br>
   &nbsp;&nbsp;&nbsp;&nbsp; Find a non trivial MVD A = {A1, A2, ..., An}&#x21A0;B in R, s.t. A is not a superkey for R

   &nbsp;&nbsp;&nbsp;&nbsp; <u>if</u> (not found) <u>then</u> Return R <br>
   
   &nbsp;&nbsp;&nbsp;&nbsp; let <b>Y = (A &cup; B)</b>,  <b>Z = A &cup; ([all attributes] \ (A &cup; B))</b> <br>
   &nbsp;&nbsp;&nbsp;&nbsp; decompose R into <b>R<sub>1</sub>(A &cup; Y)</b> and <b>R<sub>2</sub>(A &cup; Z)</b> <br>
   
   &nbsp;&nbsp;&nbsp;&nbsp; Return 4NFDecomp(R<sub>1</sub>), 4NFDecomp(R<sub>2</sub>)

### Exercise 4

For the attributes A and MVDs M (don't worry about the FDs) as defined in exercise 3, give the decomposition of the relation R into 4th normal form.